# Visualization Package
Extremely flexible tool to visualize and debug the data pipeline (data loading, processing, and augmentation ) suitable for machine learning experiments.  The package comes with a rich collection of visualizers and workflows. 

## Terminology

**sample_dict** - Represents a single sample and contains all relevant information about the sample.

No specific structure of this dictionary is required, but a useful pattern is to split it into sections (keys that define a "namespace" ): such as "data", "model",  etc.
NDict (fuse/utils/ndict.py) class is used instead of python standard dictionary in order to allow easy "." seperated access. For example:
`sample_dict[“data.input.img”]` is the equivallent of `sample_dict["data"]["input"]["img"]`

conventions :
1. you should include suffix specifying the type of the value ("img", "seg", "bbox") - which will be used by the type detector ( see VisProbe Op)
2. you can optionaly give a name to your pipeline by sample_dict["name"] = "kits21" and it will show up in every image and file created by the visualizer


## Op(erator)

Operators are the building blocks of the sample processing pipeline. Each operator gets as input the *sample_dict* as created by the previous operators and can either add/delete/modify fields in sample_dict. The operator interface is specified in OpBase class. 
A pipeline is built as a sequence of operators, which do everything - loading a new sample, preprocessing, augmentation, and more.
The visualization tool is integrated in the pipeline using the VisProbe operation

## VisProbe Op
This op can be placed between different steps at the pipeline , each step can have an optional name which will appear in every image and file created by the visualizer
This op requires the following inputs in each call :
1. give the input keys that define a "namespace" which included your input
2. type detector object of class TypeDetectorPatternsBased - used to detect the input type from previous step and convert it to the right class 
3. visualizer - instance of a class that inherits from VisualizerBase ( SaveVisual - to save to nifti files , Imaging2dVisualizer - to save multiple 2d images in one file)
4. output_path - path to save the images

## Pipeline

A sequence of operators loading, pre-processing, and augmenting a sample. We split the pipeline into two parts - static and dynamic, which allow us to control the part out of the entire pipeline that will be cached. To learn more see *Adding a dynamic part*